In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN, Bidirectional

# Load the datasets
dataset_1 = pd.read_csv('wine_red_lcm_sorted.csv')
dataset_2 = pd.read_csv('winequality-red-normalized.csv')
quality_dataset = pd.read_csv('quality.csv')

# Prepare the input combinations
inputs = [('H11', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H12', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H13', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'),
          ('H14', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H15', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H16', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11')]
# Prepare the output and convert to one-hot encoding
output = to_categorical(quality_dataset['quality2'])

# Define the number of classes
n_classes = 2

def create_model():
    model = Sequential([
        Bidirectional(SimpleRNN(units=16, activation='relu',return_sequences=False),input_shape=(9,1)),
        Dropout(0.1),
        Dense(n_classes, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['accuracy'])
    return model

models = {}  # To store all trained models
all_predictions = {}  # To store all predictions

# Loop over the input combinations, create and train a model for each
for input_tuple in inputs:
    print(f"Training model with inputs {input_tuple}...")

    # Prepare the inputs
    input_data = pd.concat([dataset_1[[input_tuple[0]]], dataset_2[list(input_tuple[1:])]], axis=1)
    input_data = input_data.values.reshape((-1, 9, 1))  # reshape input
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(input_data, output,
                                                        test_size=0.1, 
                                                        random_state=42)

    # Create and train the model
    model = create_model()
    history = model.fit(X_train, y_train,
                        batch_size=16,
                        epochs=300,
                        validation_split=0.1,
                        verbose=2)

    # Evaluate the model on the test set
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test accuracy: {accuracy}\n")

    # Generate predictions on the test set
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)

    # Store the model and predictions
    models[input_tuple] = model
    all_predictions[input_tuple] = predicted_classes  # Store the class predictions

# Initialize an empty DataFrame to store all predictions
all_predictions_df = pd.DataFrame()

# Loop over the input combinations
for input_tuple in inputs:
    print(f"Processing predictions for inputs {input_tuple}...")

    # Retrieve the predicted classes for the current input combination
    predicted_classes = all_predictions[input_tuple]

    # Convert the predictions into a DataFrame column
    predictions_df = pd.DataFrame(predicted_classes, columns=[f'{input_tuple[0]}_{input_tuple[1]}'])

    # Concatenate the current predictions with the previous predictions DataFrame
    all_predictions_df = pd.concat([all_predictions_df, predictions_df], axis=1)

# Save the all predictions DataFrame to a CSV file
all_predictions_df.to_csv('lcm_all_predictions_birnn.csv', index=False)


Training model with inputs ('H11', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x10', 'x11')...
Epoch 1/300
81/81 - 2s - loss: 0.6787 - accuracy: 0.6100 - val_loss: 0.6758 - val_accuracy: 0.6458 - 2s/epoch - 27ms/step
Epoch 2/300
81/81 - 0s - loss: 0.6641 - accuracy: 0.6595 - val_loss: 0.6608 - val_accuracy: 0.6181 - 301ms/epoch - 4ms/step
Epoch 3/300
81/81 - 0s - loss: 0.6332 - accuracy: 0.6880 - val_loss: 0.6338 - val_accuracy: 0.6181 - 292ms/epoch - 4ms/step
Epoch 4/300
81/81 - 0s - loss: 0.5816 - accuracy: 0.7305 - val_loss: 0.6273 - val_accuracy: 0.6111 - 303ms/epoch - 4ms/step
Epoch 5/300
81/81 - 0s - loss: 0.5737 - accuracy: 0.7205 - val_loss: 0.6183 - val_accuracy: 0.6528 - 293ms/epoch - 4ms/step
Epoch 6/300
81/81 - 0s - loss: 0.5582 - accuracy: 0.7375 - val_loss: 0.6063 - val_accuracy: 0.6319 - 305ms/epoch - 4ms/step
Epoch 7/300
81/81 - 0s - loss: 0.5550 - accuracy: 0.7390 - val_loss: 0.6117 - val_accuracy: 0.6389 - 290ms/epoch - 4ms/step
Epoch 8/300
81/81 - 0s - loss: 0.5510 - accura

Epoch 66/300
81/81 - 0s - loss: 0.4351 - accuracy: 0.7961 - val_loss: 0.4657 - val_accuracy: 0.7569 - 303ms/epoch - 4ms/step
Epoch 67/300
81/81 - 0s - loss: 0.4274 - accuracy: 0.8154 - val_loss: 0.5035 - val_accuracy: 0.7361 - 291ms/epoch - 4ms/step
Epoch 68/300
81/81 - 0s - loss: 0.4414 - accuracy: 0.7830 - val_loss: 0.4803 - val_accuracy: 0.7431 - 297ms/epoch - 4ms/step
Epoch 69/300
81/81 - 0s - loss: 0.4254 - accuracy: 0.8124 - val_loss: 0.4554 - val_accuracy: 0.7639 - 287ms/epoch - 4ms/step
Epoch 70/300
81/81 - 0s - loss: 0.4320 - accuracy: 0.8062 - val_loss: 0.4706 - val_accuracy: 0.7500 - 292ms/epoch - 4ms/step
Epoch 71/300
81/81 - 0s - loss: 0.4259 - accuracy: 0.8046 - val_loss: 0.4742 - val_accuracy: 0.7639 - 289ms/epoch - 4ms/step
Epoch 72/300
81/81 - 0s - loss: 0.4172 - accuracy: 0.8170 - val_loss: 0.4757 - val_accuracy: 0.7292 - 286ms/epoch - 4ms/step
Epoch 73/300
81/81 - 0s - loss: 0.4305 - accuracy: 0.8069 - val_loss: 0.4536 - val_accuracy: 0.7639 - 297ms/epoch - 4ms/step


Epoch 132/300
81/81 - 0s - loss: 0.3724 - accuracy: 0.8317 - val_loss: 0.4515 - val_accuracy: 0.7569 - 291ms/epoch - 4ms/step
Epoch 133/300
81/81 - 0s - loss: 0.3808 - accuracy: 0.8347 - val_loss: 0.4433 - val_accuracy: 0.7639 - 293ms/epoch - 4ms/step
Epoch 134/300
81/81 - 0s - loss: 0.3835 - accuracy: 0.8340 - val_loss: 0.4551 - val_accuracy: 0.7708 - 291ms/epoch - 4ms/step
Epoch 135/300
81/81 - 0s - loss: 0.3720 - accuracy: 0.8409 - val_loss: 0.4370 - val_accuracy: 0.7569 - 305ms/epoch - 4ms/step
Epoch 136/300
81/81 - 0s - loss: 0.3816 - accuracy: 0.8332 - val_loss: 0.4492 - val_accuracy: 0.7778 - 298ms/epoch - 4ms/step
Epoch 137/300
81/81 - 0s - loss: 0.3769 - accuracy: 0.8378 - val_loss: 0.4548 - val_accuracy: 0.7708 - 291ms/epoch - 4ms/step
Epoch 138/300
81/81 - 0s - loss: 0.3768 - accuracy: 0.8208 - val_loss: 0.4867 - val_accuracy: 0.7778 - 294ms/epoch - 4ms/step
Epoch 139/300
81/81 - 0s - loss: 0.3710 - accuracy: 0.8440 - val_loss: 0.4521 - val_accuracy: 0.7500 - 294ms/epoch - 4

81/81 - 0s - loss: 0.3395 - accuracy: 0.8541 - val_loss: 0.4255 - val_accuracy: 0.8056 - 294ms/epoch - 4ms/step
Epoch 198/300
81/81 - 0s - loss: 0.3521 - accuracy: 0.8510 - val_loss: 0.4637 - val_accuracy: 0.7847 - 300ms/epoch - 4ms/step
Epoch 199/300
81/81 - 0s - loss: 0.3296 - accuracy: 0.8610 - val_loss: 0.4342 - val_accuracy: 0.7847 - 292ms/epoch - 4ms/step
Epoch 200/300
81/81 - 0s - loss: 0.3360 - accuracy: 0.8486 - val_loss: 0.4447 - val_accuracy: 0.7569 - 305ms/epoch - 4ms/step
Epoch 201/300
81/81 - 0s - loss: 0.3362 - accuracy: 0.8471 - val_loss: 0.4303 - val_accuracy: 0.7917 - 290ms/epoch - 4ms/step
Epoch 202/300
81/81 - 0s - loss: 0.3432 - accuracy: 0.8494 - val_loss: 0.4252 - val_accuracy: 0.7917 - 306ms/epoch - 4ms/step
Epoch 203/300
81/81 - 0s - loss: 0.3405 - accuracy: 0.8432 - val_loss: 0.4272 - val_accuracy: 0.7708 - 304ms/epoch - 4ms/step
Epoch 204/300
81/81 - 0s - loss: 0.3287 - accuracy: 0.8510 - val_loss: 0.4131 - val_accuracy: 0.8194 - 290ms/epoch - 4ms/step
Epoch 

Epoch 263/300
81/81 - 0s - loss: 0.3161 - accuracy: 0.8641 - val_loss: 0.4730 - val_accuracy: 0.7986 - 293ms/epoch - 4ms/step
Epoch 264/300
81/81 - 0s - loss: 0.3038 - accuracy: 0.8649 - val_loss: 0.4376 - val_accuracy: 0.7778 - 295ms/epoch - 4ms/step
Epoch 265/300
81/81 - 0s - loss: 0.3040 - accuracy: 0.8633 - val_loss: 0.4306 - val_accuracy: 0.7847 - 299ms/epoch - 4ms/step
Epoch 266/300
81/81 - 0s - loss: 0.2933 - accuracy: 0.8788 - val_loss: 0.4482 - val_accuracy: 0.7986 - 292ms/epoch - 4ms/step
Epoch 267/300
81/81 - 0s - loss: 0.3129 - accuracy: 0.8556 - val_loss: 0.4551 - val_accuracy: 0.7847 - 296ms/epoch - 4ms/step
Epoch 268/300
81/81 - 0s - loss: 0.2978 - accuracy: 0.8710 - val_loss: 0.4552 - val_accuracy: 0.7708 - 294ms/epoch - 4ms/step
Epoch 269/300
81/81 - 0s - loss: 0.3072 - accuracy: 0.8687 - val_loss: 0.4218 - val_accuracy: 0.7847 - 295ms/epoch - 4ms/step
Epoch 270/300
81/81 - 0s - loss: 0.3049 - accuracy: 0.8656 - val_loss: 0.4538 - val_accuracy: 0.7778 - 297ms/epoch - 4

...

Epoch 3/300
81/81 - 0s - loss: 0.6332 - accuracy: 0.6903 - val_loss: 0.6232 - val_accuracy: 0.6458 - 305ms/epoch - 4ms/step
Epoch 4/300
81/81 - 0s - loss: 0.5911 - accuracy: 0.7305 - val_loss: 0.6036 - val_accuracy: 0.6528 - 312ms/epoch - 4ms/step
Epoch 5/300
81/81 - 0s - loss: 0.5688 - accuracy: 0.7359 - val_loss: 0.5923 - val_accuracy: 0.6736 - 313ms/epoch - 4ms/step
Epoch 6/300
81/81 - 0s - loss: 0.5616 - accuracy: 0.7243 - val_loss: 0.5856 - val_accuracy: 0.6667 - 303ms/epoch - 4ms/step
Epoch 7/300
81/81 - 0s - loss: 0.5571 - accuracy: 0.7243 - val_loss: 0.5871 - val_accuracy: 0.6597 - 302ms/epoch - 4ms/step
Epoch 8/300
81/81 - 0s - loss: 0.5462 - accuracy: 0.7375 - val_loss: 0.5763 - val_accuracy: 0.6875 - 310ms/epoch - 4ms/step
Epoch 9/300
81/81 - 0s - loss: 0.5378 - accuracy: 0.7405 - val_loss: 0.5623 - val_accuracy: 0.6875 - 307ms/epoch - 4ms/step
Epoch 10/300
81/81 - 0s - loss: 0.5362 - accuracy: 0.7514 - val_loss: 0.5580 - val_accuracy: 0.7222 - 302ms/epoch - 4ms/step
Epoch 1

Epoch 69/300
81/81 - 0s - loss: 0.4304 - accuracy: 0.8015 - val_loss: 0.4707 - val_accuracy: 0.7431 - 302ms/epoch - 4ms/step
Epoch 70/300
81/81 - 0s - loss: 0.4266 - accuracy: 0.7931 - val_loss: 0.4950 - val_accuracy: 0.7500 - 304ms/epoch - 4ms/step
Epoch 71/300
81/81 - 0s - loss: 0.4259 - accuracy: 0.7915 - val_loss: 0.4724 - val_accuracy: 0.7639 - 297ms/epoch - 4ms/step
Epoch 72/300
81/81 - 0s - loss: 0.4293 - accuracy: 0.7923 - val_loss: 0.4921 - val_accuracy: 0.7639 - 303ms/epoch - 4ms/step
Epoch 73/300
81/81 - 0s - loss: 0.4242 - accuracy: 0.7884 - val_loss: 0.4997 - val_accuracy: 0.7361 - 304ms/epoch - 4ms/step
Epoch 74/300
81/81 - 0s - loss: 0.4302 - accuracy: 0.7861 - val_loss: 0.4791 - val_accuracy: 0.7569 - 306ms/epoch - 4ms/step
Epoch 75/300
81/81 - 0s - loss: 0.4192 - accuracy: 0.7900 - val_loss: 0.4687 - val_accuracy: 0.7569 - 305ms/epoch - 4ms/step
Epoch 76/300
81/81 - 0s - loss: 0.4259 - accuracy: 0.8031 - val_loss: 0.5517 - val_accuracy: 0.7083 - 307ms/epoch - 4ms/step


Epoch 135/300
81/81 - 0s - loss: 0.3938 - accuracy: 0.8124 - val_loss: 0.4896 - val_accuracy: 0.7639 - 307ms/epoch - 4ms/step
Epoch 136/300
81/81 - 0s - loss: 0.3811 - accuracy: 0.8332 - val_loss: 0.5026 - val_accuracy: 0.7569 - 303ms/epoch - 4ms/step
Epoch 137/300
81/81 - 0s - loss: 0.3839 - accuracy: 0.8340 - val_loss: 0.4953 - val_accuracy: 0.7778 - 311ms/epoch - 4ms/step
Epoch 138/300
81/81 - 0s - loss: 0.3731 - accuracy: 0.8301 - val_loss: 0.5014 - val_accuracy: 0.7778 - 304ms/epoch - 4ms/step
Epoch 139/300
81/81 - 0s - loss: 0.3797 - accuracy: 0.8263 - val_loss: 0.4958 - val_accuracy: 0.7569 - 309ms/epoch - 4ms/step
Epoch 140/300
81/81 - 0s - loss: 0.3922 - accuracy: 0.8108 - val_loss: 0.4664 - val_accuracy: 0.7500 - 303ms/epoch - 4ms/step
Epoch 141/300
81/81 - 0s - loss: 0.3755 - accuracy: 0.8278 - val_loss: 0.5079 - val_accuracy: 0.7361 - 310ms/epoch - 4ms/step
Epoch 142/300
81/81 - 0s - loss: 0.3863 - accuracy: 0.8216 - val_loss: 0.5003 - val_accuracy: 0.7500 - 304ms/epoch - 4

81/81 - 0s - loss: 0.3526 - accuracy: 0.8394 - val_loss: 0.4511 - val_accuracy: 0.7708 - 304ms/epoch - 4ms/step
Epoch 201/300
81/81 - 0s - loss: 0.3377 - accuracy: 0.8432 - val_loss: 0.4722 - val_accuracy: 0.7569 - 299ms/epoch - 4ms/step
Epoch 202/300
81/81 - 0s - loss: 0.3538 - accuracy: 0.8332 - val_loss: 0.4997 - val_accuracy: 0.7500 - 307ms/epoch - 4ms/step
Epoch 203/300
81/81 - 0s - loss: 0.3426 - accuracy: 0.8324 - val_loss: 0.4707 - val_accuracy: 0.7778 - 304ms/epoch - 4ms/step
Epoch 204/300
81/81 - 0s - loss: 0.3507 - accuracy: 0.8394 - val_loss: 0.5022 - val_accuracy: 0.7500 - 309ms/epoch - 4ms/step
Epoch 205/300
81/81 - 0s - loss: 0.3425 - accuracy: 0.8363 - val_loss: 0.5159 - val_accuracy: 0.7361 - 314ms/epoch - 4ms/step
Epoch 206/300
81/81 - 0s - loss: 0.3385 - accuracy: 0.8409 - val_loss: 0.5511 - val_accuracy: 0.7361 - 303ms/epoch - 4ms/step
Epoch 207/300
81/81 - 0s - loss: 0.3417 - accuracy: 0.8347 - val_loss: 0.5004 - val_accuracy: 0.7569 - 303ms/epoch - 4ms/step
Epoch 

Epoch 266/300
81/81 - 0s - loss: 0.3294 - accuracy: 0.8456 - val_loss: 0.5323 - val_accuracy: 0.7431 - 328ms/epoch - 4ms/step
Epoch 267/300
81/81 - 0s - loss: 0.3241 - accuracy: 0.8394 - val_loss: 0.5284 - val_accuracy: 0.7431 - 332ms/epoch - 4ms/step
Epoch 268/300
81/81 - 0s - loss: 0.3341 - accuracy: 0.8486 - val_loss: 0.5042 - val_accuracy: 0.7431 - 299ms/epoch - 4ms/step
Epoch 269/300
81/81 - 0s - loss: 0.3393 - accuracy: 0.8394 - val_loss: 0.5202 - val_accuracy: 0.7639 - 309ms/epoch - 4ms/step
Epoch 270/300
81/81 - 0s - loss: 0.3351 - accuracy: 0.8402 - val_loss: 0.4900 - val_accuracy: 0.7431 - 360ms/epoch - 4ms/step
Epoch 271/300
81/81 - 0s - loss: 0.3233 - accuracy: 0.8440 - val_loss: 0.5418 - val_accuracy: 0.7569 - 307ms/epoch - 4ms/step
Epoch 272/300
81/81 - 0s - loss: 0.3164 - accuracy: 0.8595 - val_loss: 0.5195 - val_accuracy: 0.7292 - 304ms/epoch - 4ms/step
Epoch 273/300
81/81 - 0s - loss: 0.3245 - accuracy: 0.8432 - val_loss: 0.5203 - val_accuracy: 0.7222 - 306ms/epoch - 4

In [ ]:
all_predictions_df.to_csv('ave_all_predictions_birnn.csv', index=False)

In [ ]:
import pandas as pd

# 转化y_test为标签编码并导出
y_test_labels = np.argmax(y_test, axis=1)

# 转换为 pandas DataFrame
y_test_df = pd.DataFrame(y_test_labels, columns=['Label'])

# 保存为 CSV 文件
y_test_df.to_csv('y_test_labels.csv', index=False)


In [ ]:
inputs = [('H11', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H12', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H13', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'),
          ('H14', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H15', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11'), 
          ('H16', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11')]
inputs =[('H1', 'x3', 'x4', 'x5', 'x6', 'x7','x8', 'x10', 'x11')]